In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### Encoder

In [2]:
class VAE_Encoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            # (B, C, H, W) -> (B, 128, H, W)
            nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, padding=1),
            # (B, 128, H, W) -> (B, 128, H, W)
            VAE_ResidualBlock(in_c=128, out_c=128),
            # (B, 128, H, W) -> (B, 128, H, W)
            VAE_ResidualBlock(in_c=128, out_c=128),
            # (B, 128, H, W) -> (B, 128, H / 2, W / 2)
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding=0),
            # (B, 128, H / 2, W / 2) -> (B, 256, H / 2, W / 2)
            VAE_ResidualBlock(in_c=128, out_c=256),
            # (B, 256, H / 2, W / 2) -> (B, 256, H / 2, W / 2)
            VAE_ResidualBlock(in_c=256, out_c=256),
            # (B, 256, H / 2, W / 2) -> (B, 256, H / 4, W / 4)
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=0),
            # (B, 256, H / 4, W / 4) -> (B, 512, H / 4, W / 4)
            VAE_ResidualBlock(in_c=256, out_c=512),
            # (B, 512, H / 4, W / 4) -> (B, 512, H / 4, W / 4)
            VAE_ResidualBlock(in_c=512, out_c=512),
            # (B, 512, H / 4, W / 4) -> (B, 512, H / 8, W / 8)
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=2, padding=0),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_ResidualBlock(in_c=512, out_c=512),
            VAE_ResidualBlock(in_c=512, out_c=512),
            VAE_ResidualBlock(in_c=512, out_c=512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_AttentionBlock(512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_ResidualBlock(in_c=512, out_c=512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            nn.GroupNorm(num_groups=32, num_channels=512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            nn.SiLU(),
            # (B, 512, H / 4, W / 4) -> (B, 8, H / 8, W / 8)
            nn.Conv2d(in_channels=512, out_channels=8, kernel_size=3, padding=1),
            # (B, 8, H / 4, W / 4) -> (B, 8, H / 8, W / 8)
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1, padding=0)
        )
    
    def forward(self, x: torch.Tensor, noise: torch.Tensor) -> torch.Tensor:
        # x: (B, C, H, W) | (Image)
        # noise: (B, out_channels, H / 8, W / 8)
        for module in self:
            if getattr(module, "stride", None) == (2, 2):
                # Padding: L, R, T, B
                x = F.pad(x, (0, 1, 0, 1))
            x = module(x)
        
        # (B, 8, H / 4, W / 4) -> 2 * (B, 4, H / 8, W / 8) | Breaking the batch
        mean, log_variance = torch.chunk(x, 2, dim=1)

        # (B, 4, H / 8, W / 8) -> (B, 4, H / 8, W / 8)
        variance = torch.exp(torch.clamp(log_variance, -30, 20))
        stdev = torch.sqrt(variance)

        # Z = N(0, 1) -> N(mean, variance) = X?
        # X = mean + stdev * Z
        # Scale the output by a constant
        x = (mean + (stdev * noise)) * 0.18215

        return x

### Decoder

In [3]:
class VAE_AttentionBlock(nn.Module):
    def __init__(self, channels: int):
        super().__init__()
        self.g_norm = nn.GroupNorm(num_groups=32, num_channels=channels)
        self.attention = SelfAttention(1, channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, Features or channels, H, W)
        residue = x
        n, c, h, w = x.shape
        # (B, Features, H, W) -> (B, Features, H * W)
        x = x.view(n, c, h*w)
        # (B, Features, H * W) -> (B, H * W, Features)
        x = x.transpose(-1, -2)
        # (B, Features, H * W) -> (B, H * W, Features)
        x = self.attention(x)
        # (B, H * W, Features) -> (B, Features, H * W)
        x = x.transpose(-1, -2)
        # (B, Features, H * W) -> (B, Features, H, W)
        x = x.view(n, c, h, w)
        
        x += residue
        return x


Writing ./decoder.py


In [4]:
class VAE_ResidualBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.g_norm_1 = nn.GroupNorm(num_groups=32, num_channels=in_c)
        self.conv_1 = nn.Conv2d(in_channels=in_c, out_channels=out_c, kernel_size=3, padding=1)
        
        self.g_norm_2 = nn.GroupNorm(num_groups=in_c, num_channels=out_c)
        self.conv_2 = nn.Conv2d(in_channels=out_c, out_channels=out_c, kernel_size=3, padding=1)

        if in_c == out_c:
            self.residual_layer = nn.Identity()
        else:
            self.residual_layer = nn.Conv2d(in_channels=out_c, out_channels=out_c, kernel_size=1, padding=0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, in_channels, H, W)
        residue = x
        x = self.g_norm_1(x)
        x = F.silu(x)
        x = self.conv_1(x)
        x = self.g_norm_2(x)
        x = F.silu(x)
        x = self.conv_2(x)
        return x + self.residual_layer(residue)

In [5]:
class VAE_Decoder(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Conv2d(4, 4, kernel_size=1, padding=0),
            nn.Conv2d(4, 512, kernel_size=3, padding=1),
            VAE_ResidualBlock(512, 512),
            VAE_AttentionBlock(512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            # (B, 512, H / 8, W / 8) -> (B, 512, H / 8, W / 8)
            VAE_ResidualBlock(512, 512),
            # (B, 512, H / 8, W / 8) -> (B, 521, H / 4, W / 4)
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            VAE_ResidualBlock(512, 512),
            # (B, 512, H / 4, W / 4) -> (B, 521, H / 2, W / 2)
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            VAE_ResidualBlock(512, 256),
            VAE_ResidualBlock(256, 256),
            VAE_ResidualBlock(256, 256),
            # (B, 512, H / 2, W / 2) -> (B, 521, H, W)
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            VAE_ResidualBlock(256, 128),
            VAE_ResidualBlock(128, 128),
            VAE_ResidualBlock(128, 128),
            nn.GroupNorm(32, 128),
            nn.SiLU(),
            # (B, 128, H, W) -> (B, 3, H, W)
            nn.Conv2d(128, 3, kernel_size=3, padding=1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x /= 0.18215
        for module in self:
            x = module(x)
        return x

### Attention

In [6]:
import math
class SelfAttention(nn.Module):
    def __init__(self, n_heads: int, d_embed: int, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        self.in_proj = nn.Linear(in_features=d_embed, out_features=3 * d_embed, bias=in_proj_bias)
        self.out_proj = nn.Linear(in_features=d_embed, out_features=d_embed, bias=out_proj_bias)
        self.n_heads = n_heads
        self.d_head = d_embed // n_heads

    def forward(self, x: torch.Tensor, causual_mask=False):
        # x: (B, Seq_len, Dim)
        input_shape = x.shape
        batch_size, sequence_length, d_embed = input_shape
        interim_shape = (batch_size, sequence_length, self.n_heads, self.d_head)

        # (B, Seq_len, Dim) -> (B, Seq_len, Dim * 3) -> 3 tensors of shape (B, Seq_len, Dim)
        q, k, v = self.in_proj(x).chunk(3, dim=-1)

        # (B, Seq_len, Dim) -> (B, Seq_len, H, Dim / H) -> (B, H, Seq_len, Dim / H)
        q = q.view(interim_shape).transpose(1, 2)
        k = k.view(interim_shape).transpose(1, 2)
        v = v.view(interim_shape).transpose(1, 2)

        # (B, H, Seq_len, Seq_len)
        weight = q @ k.transpose(-1, -2)

        if causual_mask:
            # Mask where the upper triangle (above the principal diagnol) is made up of 1
            mask = torch.ones_like(weight, dtype=torch.bool).triu(1)
            weight.masked_fill(mask, -torch.inf)

        weight /= math.sqrt(self.d_head)
        weight = F.softmax(weight, dim=-1)
        # (B, H, Seq_len, Seq_len) @ (B, H, Seq_len, Dim / H) -> (B, H, Seq_len, Dim / H)
        output = weight @ v
        # (B, H, Seq_len, Dim / H) @ (B, Seq_len, H, Dim / H)
        output = output.transpose(1, 2)
        output = output.reshape(input_shape)
        output = self.out_proj(output)
        return output
